## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Fiducial Points Estimation using TAO FPENet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train a FPENet model on AFW dataset
* Run Inference on the trained model
* Export the retrained model to a .etlt file for deployment to DeepStream SDK

### Table of Contents

This notebook shows an example of Fiducial Points Estimation using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    1.1 [Verify downloaded dataset](#head-1-1) <br>
    1.2 [Download pre-trained model](#head-1-2) <br>
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Generate tfrecords from labels in json format](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Run inference for a set of images](#head-7)
8. [Deploy](#head-8)

## 0. Set up env variables <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [ ]:
# Setting up env variables for cleaner command-line commands.
# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/fpenet

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env EXPERIMENT_DIR=/results/fpenet
%env DATA_DIR=/content/drive/MyDrive/fpenet_data
# $SAMPLES_DIR is the path to the sample notebook folder and the dependency folder
# $SAMPLES_DIR/deps should exist for dependency installation
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/fpenet/specs

# Showing list of specification files.
!ls -rlt $SPECS_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

Download public dataset.

Please download and unzip the AFW dataset to `$LOCAL_EXPERIMENT_DIR` directory.

https://ibug.doc.ic.ac.uk/download/annotations/afw.zip/

### A. Download and Verify dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
# Check the dataset is present
!if [ ! -d $DATA_DIR/afw ]; then echo 'Data folder not found, please download.'; else echo 'Found Data folder.';fi

In [ ]:
# convert datset to required format
%cd /content/drive/MyDrive/ColabNotebooks/tensorflow/fpenet/
import os
from data_utils import convert_dataset
afw_data_path = os.path.join(os.environ["DATA_DIR"], 'afw')
afw_image_save_path = os.path.join(os.environ["EXPERIMENT_DIR"], 'afw')

convert_dataset(afw_data_path, os.path.join(os.environ['DATA_DIR'], 'afw/afw.json'), afw_image_save_path)
# Note that we are using dummy labels for keypoints 69 to 80.

print('Dataset conversion finished.')

In [ ]:
# Check the dataset is generated
!if [ ! -f $DATA_DIR/afw/afw.json ]; then echo 'Labels not found, please regenerate.'; else echo 'Found Labels.';fi

In [ ]:
# Sample json label.
!sed -n 1,201p $DATA_DIR/afw/afw.json

In [ ]:
# Sample image.
import os
from IPython.display import Image
Image(filename=os.path.join(afw_data_path, '134212_1.png'))

### B. Obtain pre-trained model <a class="anchor" id="head-1-2"></a>

Please follow the instructions in the following to download and verify the pretrain model for fpenet.

For FpeNet pre-trained model please download model: `nvidia/tao/fpenet:trainable_v1.0`.

After obtaining the pre-trained model, please place the model in $LOCAL_EXPERIMENT_DIR

You will then have the following path-

* pre-trained model in `$LOCAL_EXPERIMENT_DIR/pretrained_models/fpenet_vtrainable_v1.0/model.tlt`

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/content/
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /content/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/fpenet:*

In [ ]:
# Create the target destination to download the model.
!mkdir -p $EXPERIMENT_DIR/pretrained_models/

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/fpenet:trainable_v1.0 \
    --dest $EXPERIMENT_DIR/pretrained_models/

In [ ]:
!ls -rlt $EXPERIMENT_DIR/pretrained_models/fpenet_vtrainable_v1.0 

In [ ]:
# Check the model is present
!if [ ! -f $EXPERIMENT_DIR/pretrained_models/fpenet_vtrainable_v1.0/model.tlt ]; then echo 'Pretrained model file not found, please download.'; else echo 'Found Pretrain model file.';fi

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/tensorflow/setup_env.sh

### 2.4 Reset env variables <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env EXPERIMENT_DIR=/results/classification
%env DATA_DIR=/content/drive/MyDrive/tf_data/classification_data/

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/classification

%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/classification/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

## 3. Generate tfrecords from labels in json format <a class="anchor" id="head-3"></a>
* Create the tfrecords using the dataset_convert command
* Input is ground truth landmarks and output is tfrecord files

In [ ]:
# Modify dataset_config for data preparation
# verify all paths
!cat $SPECS_DIR/dataset_config.yaml

In [ ]:
!ls $DATA_DIR/afw

In [ ]:
!python3.6 -m pip install uff

In [ ]:
#!cp -r /content/drive/MyDrive/fpenet_data/afw /results/fpenet/
!ls /content/fpenet_data/data/tfrecords/afw/FpeTfRecords

In [ ]:
!fpenet dataset_convert -e $SPECS_DIR/dataset_config.yaml

In [ ]:
# check the tfrecords are generated
!if [ ! -d $EXPERIMENT_DIR/data/tfrecords/afw/FpeTfRecords ]; then echo 'Tfrecords folder not found, please generate.'; else echo 'Found Tfrecords folder.';fi

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Tfrecords for the train datasets
    * In order to use the newly generated tfrecords for training, update the 'tfrecords_directory_path' and 'tfrecord_folder_name' parameters of 'dataset_info' section in the spec file at `$SPECS_DIR/experiment_spec.yaml`
* Pre-trained model path
    * Update "pretrained_model_path" in the spec file at `$SPECS_DIR/experiment_spec.yaml`
    * If you want to train from random weights with your own data, you can enter "null" for "pretrained_model_path" section
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

In [ ]:
!cat $SPECS_DIR/experiment_spec.yaml

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models

*Note: The training may take hours to complete. Also, the remaining notebook, assumes that the training was done in single-GPU mode. 



In [ ]:
!fpenet train -e $SPECS_DIR/experiment_spec.yaml \
                  -r $EXPERIMENT_DIR/models/exp1 \
                  -k $KEY

In [ ]:
# check the training folder for generated files
!ls -lh $LOCAL_EXPERIMENT_DIR/models/exp1

## 6. Evaluate the trained model <a class="anchor" id="head-6"></a>


In [ ]:
!tao fpenet evaluate  -m $USER_EXPERIMENT_DIR/models/exp1 \
                      -k $KEY

In [ ]:
# check the kpi predictions file is generated
!if [ ! -f $LOCAL_EXPERIMENT_DIR/models/exp1/kpi_testing_error_per_region.csv ]; then echo 'KPI results file not found!'; else cat $LOCAL_EXPERIMENT_DIR/models/exp1/kpi_testing_error_per_region.csv;fi
# Since keypoints 69 to 80 are dummy labels, error for pupil and ears would be high.

## 7. Run inference on testing set <a class="anchor" id="head-7"></a>

In [ ]:
!tao fpenet inference -e $SPECS_DIR/experiment_spec.yaml \
                      -i $SPECS_DIR/inference_sample.json \
                      -r $LOCAL_PROJECT_DIR \
                      -m $USER_EXPERIMENT_DIR/models/exp1/model.tlt \
                      -o $USER_EXPERIMENT_DIR/models/exp1 \
                      -k $KEY

In [ ]:
# check the results file is generated
!if [ ! -f $LOCAL_EXPERIMENT_DIR/models/exp1/result.txt ]; then echo 'Results file not found!'; else cat $LOCAL_EXPERIMENT_DIR/models/exp1/result.txt;fi

In [ ]:
import os
import cv2
import IPython.display
import PIL.Image
%matplotlib inline
# read results
results_file = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], 'models/exp1/result.txt')
results = open(results_file, 'r').readlines()[0] # display one image as an example

pred_part = results.strip().split(' ')
# get image path (append root path, if present)
image_path = pred_part[0].replace(os.environ["USER_EXPERIMENT_DIR"], os.environ["LOCAL_EXPERIMENT_DIR"])
# get predictions
fl_res = [float(x) for x in pred_part[1:]]
# read image
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# set color for landmarks
fl_color=(0,255,0)
# loop through keypoints and draw on image
for q in range(76): # not drawing ear points
    row_pred_x = fl_res[2*q]
    col_pred_y = fl_res[(2*q)+1]
    img_rgb = cv2.circle(img_rgb,(int(row_pred_x), int(col_pred_y)), 1, fl_color, 1)
# display image
IPython.display.display(PIL.Image.fromarray(img_rgb))
# Note that the accuracy is not gauranteed for this visualization example.